# Rôle de _footprints_ dans la création des objets

Reprenons l'exemple précédent de création d'une *ressource* :

```python
pe_resource = fp.proxy.resource(kind='gridpoint',
                                term=3,
                                geometry='euroc25',
                                nativefmt='grib',
                                model='arpege',
                                cutoff='production',
                                date='2017060118',
                                origin='historic')
```

On appelle une méthode générique qui crée le "bon" objet en fonction des attributs qui lui sont passés : cela fait fortement penser au patron de conception de la *Fabrique* (*factory design pattern* pour les anglophones). C'est une façon de décrire le rôle du package **footprints**, néanmoins **footprints** fait bien plus que d'implémenter une Fabrique (notamment en terme de flexibilité).

## Description des objets gérés par _footprints_

Nous allons maintenant commencer notre démonstration du package footprints, la moindre des choses est de charger le package :

In [1]:
import footprints as fp

En soi, le fait de charger le package **footprints** ne fait rien. Tout se joue lors de l'écriture des classes concernées et de la création des objets.

### Classes, Collecteurs et Objets

Voici un exemple de classe adossée au package _footprints_ :

In [2]:
class Observations(fp.FootprintBase):
    
    _collector = ('fakeresource', )  # collecteur
    _footprint = dict(  # empreinte
        info = 'Any kind of insitu observations',
        attr = dict(
            kind = dict(
                values = ['observations', ],
                info   = 'The kind of resource',
            )
        )
    )

* Toute classe "footprintisée" doit hériter de **FootprintBase**
* Une classe "footprintisée" est rattachée à un (ou plusieurs) *collecteur*. Chaque collecteur est un objet de classe **footprints.collectors.Collector** qui maintient, pour un thème donné (ici "fakeresource"), une liste des différentes classes qui ont été déclarées.
* Chaque classe "footprintisée" doit être munie d'une empreinte qui spécifie quels sont les attributs d'une classe, ...

La récupération d'un objet collecteur peut se faire via le module **collectors** :

In [3]:
print(fp.collectors.get(tag='fakeresource'))

En général, une autre syntaxe est préférée :

In [4]:
from footprints import proxy as fpx
print(fpx.fakeresources)  # Attention au "s" à la fin

Quelles sont les classes rattachées au collecteur "fakeresource" ?

In [5]:
print([r for r in fpx.fakeresources])

[<class '__main__.Observations'>]


On retrouve bien la classe que nous avons définie précédemment...

En tant que catalogue des différentes classes disponibles, c'est le *collecteur* qui permet la création d'objets :

In [6]:
obs_obj1 = fpx.fakeresources.load(kind='observations')
print(obs_obj1)

<__main__.Observations object at 0x7f0f0815b6d8 | footprint=1>


Là aussi, le package **footprints** fournit une syntaxe simplifiée qui est souvent préférée à la précédente :

In [7]:
obs_obj2 = fpx.fakeresource(kind='observations')  # Attention : pas de "s" à la fin
print(obs_obj2)

<__main__.Observations object at 0x7f0f0815b710 | footprint=1>


À partir de ce simple exemple on peut entrevoir l'éventualité d'erreurs... Que se passe-t-il si l'on demande un objet dont le *kind* n'existe pas...

In [8]:
failer1 = fpx.fakeresource(kind='toto')
print('failer1 is:', str(failer1))

# [2019/01/17-11:19:19][footprints.collectors][pickup_and_cache:0296][WARNING]: No fakeresource found in description 
      dict(
          fakeresource = None, 
          kind = 'toto',
      )



 Report Footprint-Fakeresource: 

     __main__.Observations
         kind       : {'args': 'toto', 'why': 'Not in values'}

failer1 is: None


En cas d'erreur, le *collecteur* renvoie *None* et un message d'erreur est affiché détaillant pourquoi il n'est pas possible d'instancier telle ou telle classe.

### Héritage et footprints prédéfinies

Dans un langage comme Python, l'utilisation de l'héritage de classe est plus que recommandée. Il faut donc que le package **footprints** gère ce cas de façon optimale. En pratique, toute classe héritant d'une classe "footprintisée" hérite aussi de l'empreinte de la classe mère. Cette empreinte peut ensuite être altérée ou étendue au niveau de la classe fille.

**footprints** offre aussi la possibilité de créer des classes abstraites. Cela permet de définir des classes indispensables à la structure de l'arbre d'héritage tout en ayant la certitude que celles-ci ne seront jamais instanciées.

Certains attributs sont des "classiques", on peut imaginer de les prédéfinir.

Dans un contexte de prévision numérique, on peut par exemple considérer que le *cutoff* est un attribut classique (assimilation ou production) et vouloir le prédéfinir :

In [9]:
a_cutoff = dict(
    info     = "The cutoff type of the generating process.",
    values   = ['assim', 'production', ],
)
cutoff = fp.Footprint(info = 'Abstract cutoff', attr = dict(cutoff = a_cutoff))

Cet attribut prédéfini peut dès lors être utilisé pour définir une classe abstraite servant de base à différents champs modèle :

In [10]:
class ModelState(fp.FootprintBase):

    _collector = ('fakeresource', )
    _abstract = True
    _footprint = [
        cutoff,
        dict(
            info = 'Abstract Model State',
            attr = dict(
                kind = dict(
                    info = 'The kind of modelstates we are dealing with',
                ),
                date = dict(
                    info = 'The base date of this model state'
                ),
            )
        )
    ]


Héritons de cette classe pour créer des conditions initiales :

In [11]:
class InitialConditions(ModelState):
    
    _footprint = dict(
        attr = dict(
            kind = dict(
                values = ['initial_conditions', ]
            ),       
        )
    )    

In [12]:
ic_obj = fpx.fakeresource(kind='initial_conditions', date='2017070100', cutoff='assim')
print(ic_obj)
print('kind={0.kind:s}, date={0.date:s}, cutoff={0.cutoff:s}'.format(ic_obj))

<__main__.InitialConditions object at 0x7f0f0815be10 | footprint=3>
kind=initial_conditions, date=2017070100, cutoff=assim


Les footprints prédéfinies sont très utiles pour définir des attributs, mais on aurait envie d'aller plus loin et d'altérer le fonctionnement de la classe suite à l'ajout de l'attribut. Exemple concret :

* Ajouter un attribut *cutoff* (c'est ce que l'on a fait précédemment)
* Ajouter à la classe une property permettant d'obtenir une version abrégé du nom de cutoff (`assim` étant abrégé par `A`, `production` par `P`)

Pour cela, on va associer un **décorateur de classe** à l'attribut footprints prédéfini. Ce décorateur sera esuite appliqué par *footprints* au moment de la création de la classe. Voici le décorateur :

In [13]:
def _cutoffbis_deco(cls):
    def _get_abbrev_cutoff(self):
        return dict(assim='A', production='P').get(self.cutoff, 'X')
    cls.abbrev_cutoff = property(_get_abbrev_cutoff, doc="Abbreviated cutoff name.")
    return cls

On associe l'attribut prédéfini *cutoff* au décorateur nouvellement créé :

In [14]:
cutoff_deco = fp.DecorativeFootprint(cutoff,
                                     decorator = [_cutoffbis_deco, ])

Utilisons ce **DecorativeFootprint** :

In [15]:
class PerturedModelState(ModelState):
    
    _footprint = [cutoff_deco,
        dict(
            attr = dict(
                kind = dict(
                    values = ['perturbed_state', ]
                ),       
            )
        )]

La classe résultante possède bien une *property* `abbrev_cutoff` (ajoutée par le décorateur) :

In [16]:
ps_obj = fpx.fakeresource(kind='perturbed_state', date='2017070100', cutoff='production')
print(ps_obj)
print('kind={0.kind:s}, date={0.date:s}, cutoff={0.cutoff:s}, abbrev_cutoff={0.abbrev_cutoff:s}'.format(ps_obj))

<__main__.PerturedModelState object at 0x7f0f0817f4e0 | footprint=3>
kind=perturbed_state, date=2017070100, cutoff=production, abbrev_cutoff=P


### Description détaillée des attributs

Il est possible d'utiliser un grand nombre de clés lors de la description des attributs :

  * **type** : type (au sens de Python) de l'attribut (*str* par défaut)
  * **outcast** : opposé de *values* : on précise les valeurs interdites et non celles autorisées
  * **optional** : un attribut peut être optionnel (par défaut un attribut est obligatoire)
  * **default** : si ``optional=True`` on peut préciser une valeur par défaut (*None* par défaut)
  * **alias** : lors de la phase de création des objets (résolution des empreintes), un même attribut peut avoir plusieurs synonymes
  * **remap** : il est possible de faire pointer une valeur autorisée vers une autre



In [17]:
class Historic(ModelState):
    
    _footprint = dict(
        attr = dict(
            kind = dict(
                values = ['historic', 'forecastfile', ],
                remap = dict(forecastfile='historic', ),
            ),
            term = dict(
                info = "The forecast's term",
                type = int,
                alias = ('fcterm', 'forecastterm', ),
            ),
            nativefmt = dict(
                info = "The storage's format.",
                values = ['grib', 'fa', 'netcdf', ],
                optional = True,
                default = 'grib',
            ),
        )
    )

#### Démonstration de *remap* et *optional*

Rappel:

  * pour *kind*, ``remap = dict(forecastfile='historic', )`` ;
  * pour *nativefmt*, ``optional = True`` et ``default = 'grib'``.

In [18]:
obj_hst1 = fpx.fakeresource(kind='forecastfile',
                            cutoff='assim', date='20170701',
                            term=6)
print(obj_hst1, "object's kind:     ", obj_hst1.kind)
print(obj_hst1, "object's nativefmt:", obj_hst1.nativefmt)

<__main__.Historic object at 0x7f0f08187c18 | footprint=5> object's kind:      historic
<__main__.Historic object at 0x7f0f08187c18 | footprint=5> object's nativefmt: grib


#### Démonstration de *alias*

Rappel: pour *term*, ``alias = ('fcterm', 'forecastterm', ),``.

In [19]:
obj_hst1 = fpx.fakeresource(kind='forecastfile',
                            cutoff='assim', date='20170701',
                            fcterm=6)
print(obj_hst1, "object's term:  ", obj_hst1.term)
try:
    print(obj_hst1, "object's fcterm:", obj_hst1.fcterm)
except AttributeError:
    print("Notice that 'fcterm' does not really exist.")

<__main__.Historic object at 0x7f0f0c767d30 | footprint=5> object's term:   6
Notice that 'fcterm' does not really exist.


#### Démonstration de *type*

Rappel: pour *term*, ``type = int,``

In [20]:
# Ok
obj_hst1 = fpx.fakeresource(kind='forecastfile', cutoff='assim', date='20170701',
                            term='6')
print("{0!r} object's term is {0.term:d}\n".format(obj_hst1))
# KO
print('This will fail:')
obj_hst1 = fpx.fakeresource(kind='forecastfile', cutoff='assim', date='20170701',
                            term='This is not an integer !')

# [2019/01/17-11:19:20][footprints.collectors][pickup_and_cache:0296][WARNING]: No fakeresource found in description 
      dict(
          cutoff = 'assim', 
          date = '20170701', 
          fakeresource = None, 
          kind = 'forecastfile', 
          term = 'This is not an integer !',
      )


<__main__.Historic object at 0x7f0f0815bbe0> object's term is 6

This will fail:

 Report Footprint-Fakeresource: 

     __main__.Historic
         term       : {'args': ('int', 'This is not an integer !'), 'why': 'Could not reclass'}

     __main__.InitialConditions
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}

     __main__.Observations
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}

     __main__.PerturedModelState
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}



Ici nous avons utilisé un type simple qui préexistait dans le langage Python (*int*), il est bien sûr possible d'utiliser des types personnalisés (la seule contrainte est que le type utilisé doit être "hashable").

C'est d'ailleurs ce que nous aurions dû faire pour les attributs *date* et *term*. En effet :

  * une date n'est pas une simple chaîne de caractères (il faut vérifier que la date fournie est valide),
  * une échéance n'est malheureusement pas un entier mais peut être exprimée en heures et minutes.
  
S'il y a lieu, l'usage de types personnalisés est donc fortement recommandé (voir par exemple les classes **bronx.stdtypes.date.Date** et **bronx.stdtypes.date.Time**)

## Notion de valeurs prédéfinies

Il est souvent utile de préciser, en début de script, des valeurs pré-définies pour un certain nombre d'attributs. Dans nos exemples précédents, on imagine assez bien que l'utilisateur puisse vouloir fixer une *date* et un *cutoff*.

Il suffit de définir ces valeurs via l'objet **setup** du package **footprints**.

NB: Il ne s'agit que de valeurs prédéfinies : si l'utilisateur précise une autre valeur lors de la création de l'objet, celle-ci sera prioritaire.

Définissons un *cutoff* et une *date* par défaut :

In [21]:
fp.setup.defaults =  dict(cutoff='assim', date='20170701')

# Utilisation du defaut:
obj_hst2a = fpx.fakeresource(kind='forecastfile', term=6)
print('{0!r}: cutoff={0.cutoff:s}, date={0.date:s}'.format(obj_hst2a))
# Defaut redefini manuellement pour le cutoff:
obj_hst2b = fpx.fakeresource(kind='forecastfile', cutoff='production', term=6)
print('{0!r}: cutoff={0.cutoff:s}, date={0.date:s}'.format(obj_hst2b))

<__main__.Historic object at 0x7f0f0815b390>: cutoff=assim, date=20170701
<__main__.Historic object at 0x7f0f0815b2e8>: cutoff=production, date=20170701


## Conclusion

Lors de la création de classes utilisant **footprints** :

* Bien recourir à l'héritage et déclarer comme telles les classes abstraites ;
* Pour les attributs les plus courants, définir des empreintes pré-définies et les utiliser ;
* En sus des empreintes pré-définies, définir des décorateurs de classe permettant d'altérer le fonctionnement des classes en cohérence avec le/les attributs pré-définis
* Penser à préciser des clés "info" qui sont facultatives mais apparaissent dans la documentation automatique  ;
* Décrire les types de données le plus précisément possible.

Questions éventuelles : *vortex.support@meteo.fr*